# Simple UI (Gradio) — Phrase → Story → Image + Score
This loads the **story model from the cloud** (Hugging Face), then:
1. Generates a story from your input phrase
2. Generates an image for that story
3. Computes and displays CLIPScore



In [1]:
# !pip -q install gradio transformers diffusers accelerate safetensors pillow torch torchvision torchaudio

import os, time, tempfile
from pathlib import Path
from PIL import Image
import torch
import gradio as gr

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline as hf_pipeline, CLIPProcessor, CLIPModel
from diffusers import StableDiffusionPipeline

# ---- Config ----
TEXT_MODEL_ID = "PrinceRansom7/story-generator-distilgpt2"  # << set this to your HF Hub repo id
GUIDANCE_SCALE = 7.0
NUM_STEPS = 15
MAX_STORY_TOKENS = 200
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Lazy globals (load once) ----
_story_pipe = None
_sd_pipe = None
_clip_model = None
_clip_proc = None

def load_pipelines():
    global _story_pipe, _sd_pipe, _clip_model, _clip_proc
    if _story_pipe is None:
        tok = AutoTokenizer.from_pretrained(TEXT_MODEL_ID)
        txt_model = AutoModelForCausalLM.from_pretrained(TEXT_MODEL_ID).to(device)
        _story_pipe = hf_pipeline("text-generation", model=txt_model, tokenizer=tok, device=0 if device=="cuda" else -1)
    if _sd_pipe is None:
        _sd_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16 if device=="cuda" else torch.float32).to(device)
    if _clip_model is None:
        _clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    if _clip_proc is None:
        _clip_proc = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def run_pipeline(phrase):
    load_pipelines()

    # 1) story
    story = _story_pipe(phrase, max_length=MAX_STORY_TOKENS, do_sample=True, top_p=0.95, temperature=0.9)[0]["generated_text"]

    # 2) image
    with torch.autocast(device) if device=="cuda" else torch.no_grad():
        img = _sd_pipe(prompt=story, guidance_scale=GUIDANCE_SCALE, num_inference_steps=NUM_STEPS).images[0]

    # 3) score
    inputs = _clip_proc(text=[story], images=img, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        logits_per_image = _clip_model(**inputs).logits_per_image
        score = logits_per_image.softmax(dim=-1)[0,0].item()

    # save temp image file for UI
    tmp = Path(tempfile.mkdtemp()) / "gen.png"
    img.save(tmp)

    return story, Image.open(tmp), float(score)

with gr.Blocks() as demo:
    gr.Markdown("# Phrase → Story → Image (with CLIPScore)")
    gr.Markdown("Enter a short phrase. The app will generate a story, create an image based on the story, and compute a similarity score.")
    inp = gr.Textbox(label="Input phrase", placeholder="e.g., A lighthouse on a stormy night")
    btn = gr.Button("Generate")
    story_out = gr.Textbox(label="Generated story")
    img_out = gr.Image(label="Generated image")
    score_out = gr.Number(label="CLIPScore (≈ matching)")

    btn.click(run_pipeline, inputs=inp, outputs=[story_out, img_out, score_out])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f782618bc7476858d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  0%|          | 0/15 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f782618bc7476858d1.gradio.live
